In [1]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score, root_mean_squared_error


C:\Users\Bernhard\.conda\envs\tf_new\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [2]:
with open('data/curated/player_data.json', 'r') as f:
    player_data = json.load(f)

In [3]:
df = pd.DataFrame(player_data)

In [4]:
df = df.astype({
    'match_id': 'string',
    'account_id': 'string',
    'hero_id': 'string',
    'radiant_team_id': 'string',
    'dire_team_id': 'string',
    'win': 'boolean',
    'firstblood_claimed': 'boolean',
    'team_id': 'string',
    'enemy_team_id': 'string',
    'valveId': 'string',
})

In [5]:
q1_glicko = df['glicko2_rating'].quantile(0.25)
df['glicko2_rating'] = df['glicko2_rating'].fillna(q1_glicko)
q1_enemy_glicko = df['enemy_glicko2_rating'].quantile(0.25)
df['enemy_glicko2_rating'] = df['enemy_glicko2_rating'].fillna(q1_enemy_glicko)

In [6]:
mask = df['rating_advantage'].isna()
df.loc[mask, 'rating_advantage'] = df.loc[mask, 'glicko2_rating'] - df.loc[mask, 'enemy_glicko2_rating']

In [7]:
df = df.sort_values(['account_id', 'match_start_time'])


In [8]:
SEQ_LEN=5

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15437 entries, 1126 to 14854
Data columns (total 46 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   match_id                  15437 non-null  string 
 1   match_start_time          15437 non-null  int64  
 2   account_id                15437 non-null  string 
 3   name                      15437 non-null  object 
 4   fantasy_role              15437 non-null  object 
 5   hero_id                   15437 non-null  string 
 6   hero_variant              15437 non-null  object 
 7   isRadiant                 15437 non-null  bool   
 8   radiant_team_id           15437 non-null  string 
 9   dire_team_id              15437 non-null  string 
 10  tournament_start_date     15437 non-null  object 
 11  duration                  15437 non-null  int64  
 12  win                       15437 non-null  boolean
 13  kills                     15437 non-null  int64  
 14  deaths  

In [10]:
not_features = [
    'match_id', 'name', 'match_start_time', 'radiant_team_id', 'dire_team_id',
    'tournament_start_date', 'teamName', 'valveId', 'enemy_teamName', 'enemy_valveId', 'fantasy_score',
    'rolling_winrate_10',
    'rolling_winrate_15',
    'rolling_fantasy_score_10',
    'rolling_fantasy_score_15',
    'rolling_enemy_glicko2_10',
    'rolling_enemy_glicko2_15',
]

In [11]:
not_features_fe = [
    'match_id', 'name', 'match_start_time', 'radiant_team_id', 'dire_team_id',
    'tournament_start_date', 'teamName', 'valveId', 'enemy_teamName', 'enemy_valveId', 'fantasy_score',
]

In [12]:
df['isRadiant'] = df['isRadiant'].astype(int)
df['win'] = df['win'].astype(int)
df['firstblood_claimed'] = df['firstblood_claimed'].astype(int)

In [13]:
df = pd.get_dummies(df, columns=['fantasy_role', 'hero_id', 'hero_variant'])

# LSTM without FE

In [14]:
#features = [col for col in df.columns if col not in not_features]
features = [col for col in df.columns if col not in not_features and pd.api.types.is_numeric_dtype(df[col])]#df[features].isna().sum()
df[features] = df[features].fillna(df[features].mean())

In [15]:
scaler = StandardScaler()
df[features] = scaler.fit_transform(df[features])

Create sequences

In [16]:
X_seqs = []
y_seqs = []

for _, player_df in df.groupby('account_id'):
    player_df = player_df.reset_index(drop=True)
    for i in range(len(player_df) - SEQ_LEN):
        X_seqs.append(player_df.loc[i:i+SEQ_LEN-1, features].values)
        y_seqs.append(player_df.loc[i+SEQ_LEN, 'fantasy_score'])

X_seqs = np.array(X_seqs)
y_seqs = np.array(y_seqs)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X_seqs, y_seqs, test_size=0.20, random_state=1001)

In [18]:
early_stop = EarlyStopping(
    monitor='val_loss',      
    patience=3,
    restore_best_weights=True
)

In [19]:
model = models.Sequential([
    layers.Input(shape=(SEQ_LEN, X_train.shape[2])),
    layers.LSTM(256, return_sequences=True),
    layers.Dropout(0.3),
    layers.LSTM(256, return_sequences=True),
    layers.LSTM(128, return_sequences=True),
    layers.LSTM(64, return_sequences=False),
    layers.Dense(1)
])
model.compile(optimizer='adam', loss='mse')
model.summary()

history = model.fit(
    X_train, y_train,
    epochs=30,
    batch_size=32,
    validation_split=0.1,
    callbacks=[early_stop]
)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 5, 256)         │       705,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 5, 256)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 5, 256)         │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 5, 128)         │       197,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,477,441 (5.64 MB)

 Trainable params: 1,477,441 (5.64 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
324/324 ━━━━━━━━━━━━━━━━━━━━ 9s 17ms/step - loss: 66.9872 - val_loss: 24.4028
Epoch 2/30
324/324 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - loss: 24.1399 - val_loss: 22.5997
Epoch 3/30
324/324 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - loss: 22.6809 - val_loss: 22.2168
Epoch 4/30
324/324 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - loss: 21.5099 - val_loss: 22.3491
Epoch 5/30
324/324 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - loss: 21.2690 - val_loss: 21.9854
Epoch 6/30
324/324 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - loss: 20.7473 - val_loss: 22.0648
Epoch 7/30
324/324 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - loss: 20.0130 - val_loss: 23.2923
Epoch 8/30
324/324 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - loss: 18.1506 - val_loss: 23.6674


# LSTM with FE

In [20]:
features_fe = [col for col in df.columns if col not in not_features and pd.api.types.is_numeric_dtype(df[col])]#df[features].isna().sum()
df[features_fe] = df[features_fe].fillna(df[features_fe].mean())

In [21]:
scaler = StandardScaler()
df[features_fe] = scaler.fit_transform(df[features_fe])

In [22]:
X_seqs_fe = []
y_seqs_fe = []

for _, player_df in df.groupby('account_id'):
    player_df = player_df.reset_index(drop=True)
    for i in range(len(player_df) - SEQ_LEN):
        X_seqs_fe.append(player_df.loc[i:i+SEQ_LEN-1, features_fe].values)
        y_seqs_fe.append(player_df.loc[i+SEQ_LEN, 'fantasy_score'])

X_seqs_fe = np.array(X_seqs_fe)
y_seqs_fe = np.array(y_seqs_fe)

In [23]:
X_train_fe, X_test_fe, y_train_fe, y_test_fe = train_test_split(X_seqs_fe, y_seqs_fe, test_size=0.20, random_state=1001)

In [24]:
model_fe = models.Sequential([
    layers.Input(shape=(SEQ_LEN, X_train_fe.shape[2])),
    layers.LSTM(256, return_sequences=True),
    layers.Dropout(0.3),
    layers.LSTM(256, return_sequences=True),
    layers.LSTM(128, return_sequences=True),
    layers.LSTM(64, return_sequences=False),
    layers.Dense(1)
])
model_fe.compile(optimizer='adam', loss='mse')
model_fe.summary()

history = model_fe.fit(
    X_train_fe, y_train_fe,
    epochs=30,
    batch_size=32,
    validation_split=0.1,
    callbacks=[early_stop]
)

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_4 (LSTM)                   │ (None, 5, 256)         │       705,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 5, 256)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, 5, 256)         │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_6 (LSTM)                   │ (None, 5, 128)         │       197,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,477,441 (5.64 MB)

 Trainable params: 1,477,441 (5.64 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
324/324 ━━━━━━━━━━━━━━━━━━━━ 9s 17ms/step - loss: 70.0440 - val_loss: 24.3701
Epoch 2/30
324/324 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - loss: 23.9663 - val_loss: 24.0717
Epoch 3/30
324/324 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - loss: 23.8033 - val_loss: 22.2557
Epoch 4/30
324/324 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - loss: 21.5806 - val_loss: 22.5464
Epoch 5/30
324/324 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - loss: 21.0621 - val_loss: 21.8930
Epoch 6/30
324/324 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - loss: 19.8600 - val_loss: 22.1805
Epoch 7/30
324/324 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - loss: 20.2832 - val_loss: 22.9428
Epoch 8/30
324/324 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - loss: 17.9053 - val_loss: 24.4092


# Comparison LSTM with/without FE

In [25]:
y_pred = model.predict(X_test)
print("LSTM without FE, RMSE:", root_mean_squared_error(y_test, y_pred))

90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
LSTM without FE, RMSE: 4.619259505049918


In [26]:
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

360/360 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


In [27]:
print("LSTM without FE, Train RMSE:", root_mean_squared_error(y_train, y_train_pred))
print("LSTM without FE, Test RMSE:", root_mean_squared_error(y_test, y_test_pred))
print("LSTM without FE, Train MAE:", mean_absolute_error(y_train, y_train_pred))
print("LSTM without FE, Test MAE:", mean_absolute_error(y_test, y_test_pred))
print("LSTM without FE, Train R2:", r2_score(y_train, y_train_pred))
print("LSTM without FE, Test R2:", r2_score(y_test, y_test_pred))

LSTM without FE, Train RMSE: 4.532761162325099
LSTM without FE, Test RMSE: 4.619259505049918
LSTM without FE, Train MAE: 3.5711182265205172
LSTM without FE, Test MAE: 3.6660958325519175
LSTM without FE, Train R2: 0.1419711929342954
LSTM without FE, Test R2: 0.08877151613823142


In [28]:
y_pred_fe = model_fe.predict(X_test_fe)
print("LSTM with FE, RMSE:", root_mean_squared_error(y_test_fe, y_pred_fe))

90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step
LSTM with FE, RMSE: 4.627726738316629


In [29]:
print("LSTM with FE, Train RMSE:", root_mean_squared_error(y_train, y_train_pred))
print("LSTM with FE, Test RMSE:", root_mean_squared_error(y_test, y_test_pred))
print("LSTM with FE, Train MAE:", mean_absolute_error(y_train, y_train_pred))
print("LSTM with FE, Test MAE:", mean_absolute_error(y_test, y_test_pred))
print("LSTM with FE, Train R2:", r2_score(y_train, y_train_pred))
print("LSTM with FE, Test R2:", r2_score(y_test, y_test_pred))

LSTM with FE, Train RMSE: 4.532761162325099
LSTM with FE, Test RMSE: 4.619259505049918
LSTM with FE, Train MAE: 3.5711182265205172
LSTM with FE, Test MAE: 3.6660958325519175
LSTM with FE, Train R2: 0.1419711929342954
LSTM with FE, Test R2: 0.08877151613823142


In [30]:
def make_symmetric_matchups(matchups):
    symmetric = []
    for t1, t2 in matchups:
        symmetric.append((t1, t2))
        symmetric.append((t2, t1))
    return symmetric

In [31]:
def predict_players_for_matchups_lstm(df, model, matchup_list, features, seq_len=10, lineup_dir="data/lineups"):
    results = []
    for team_id, enemy_team_id in matchup_list:
        lineup_path = os.path.join(lineup_dir, f"team_lineup_{team_id}.json")
        try:
            with open(lineup_path, "r", encoding="utf-8") as f:
                lineup = [p for p in json.load(f) if p.get("is_current_team_member")]
        except FileNotFoundError:
            continue

        for player in lineup:
            account_id = str(player["account_id"])
            player_hist = df[df["account_id"] == account_id].sort_values("match_start_time")
            if len(player_hist) < seq_len:
                continue

            seq_df = player_hist.iloc[-seq_len:].copy()
            seq_df["team_id"] = team_id
            seq_df["enemy_team_id"] = enemy_team_id

            # Preprocessing (same as training)
            seq_df['isRadiant'] = seq_df['isRadiant'].astype(int)
            seq_df['win'] = seq_df['win'].astype(int)
            seq_df['firstblood_claimed'] = seq_df['firstblood_claimed'].astype(int)
            for col in features:
                if col not in seq_df.columns:
                    seq_df[col] = 0
            seq_df = seq_df[features]
            seq_df = seq_df.fillna(seq_df.mean()).astype(np.float32)
            print(seq_df.head())
            print(seq_df.sum())
            X_pred = seq_df.values.reshape(1, seq_len, len(features))
            pred_score = model.predict(X_pred)[0][0]
            results.append({
                "account_id": account_id,
                "name": player.get("name", ""),
                "team_id": team_id,
                "enemy_team_id": enemy_team_id,
                "predicted_fantasy_score": pred_score
            })
    return pd.DataFrame(results)

In [32]:
matchups = [('7119388', '8261500'), ('8597976', '7119388'), ('8597976', '8261500')] # Spirit vs. Xtreme Gaming, Spirit vs. Talon
symmetric_matchups = make_symmetric_matchups(matchups)
result = predict_players_for_matchups_lstm(df, model, symmetric_matchups, features, seq_len=SEQ_LEN)

       isRadiant  duration  win     kills    deaths  last_hits    denies  \
14628        0.0  0.375811  0.0 -0.705398  0.388797  -1.141037 -0.654243   
14623       -1.0  0.829555  0.0 -0.449908 -0.267467  -0.738604 -0.495595   
14568        0.0 -0.025864  0.0 -1.216377  1.045062  -1.085909 -0.971540   
14558        0.0  0.042569  0.0 -0.449908 -0.267467  -1.074883 -1.130188   
14548        0.0 -0.405224  0.0  0.061071 -0.267467  -1.201677 -0.971540   

       gold_per_min  towers_killed  roshans_killed  ...  hero_variant_95, 2  \
14628     -1.203780      -0.642407        1.492757  ...           -0.030129   
14623     -0.644496      -0.642407       -0.425313  ...           -0.030129   
14568     -0.964087      -0.046622        1.492757  ...           -0.030129   
14558     -0.907017      -0.642407       -0.425313  ...           -0.030129   
14548     -1.043984      -0.642407       -0.425313  ...           -0.030129   

       hero_variant_96, 1  hero_variant_96, 2  hero_variant_97, 2  \

In [33]:
result

,account_id,name,team_id,enemy_team_id,predicted_fantasy_score
0,113331514,Miposhka,7119388,8261500,15.560166
1,321580662,Yatoro,7119388,8261500,15.911960
2,302214028,Collapse,7119388,8261500,12.140779
3,106305042,Larl,7119388,8261500,15.861832
4,847565596,rue,7119388,8261500,15.384741
5,203351055,Malik,7119388,8261500,14.332089
6,137129583,Xm,8261500,7119388,16.140188
7,157475523,XinQ,8261500,7119388,15.266684
8,129958758,Xxs,8261500,7119388,12.367988
9,898754153,Ame,8261500,7119388,16.066925


In [34]:
#print(np.std(X_test, axis=0)) 

In [35]:
print(np.mean(y_train))

14.64377019147309


In [36]:
y_train[0]

np.float64(12.0488484)

In [37]:
y_train[1]

np.float64(9.06261538)

In [38]:
y_pred = model.predict(X_test)
print(y_pred[:10])

90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
[[14.624593 ]
 [15.970127 ]
 [15.957687 ]
 [14.430693 ]
 [13.493705 ]
 [15.424627 ]
 [15.5490675]
 [15.754745 ]
 [15.497242 ]
 [12.725742 ]]
